<div class="alert alert-block alert-info">
    This script <b> calculates transgression levels for multiple PBs and adjusts prices from <code>EUR2005</code> to <code>USD2018</code></b>.
    <hr> 
    Note: <br>
    <i><b>Input file(s)' name(s) and metadata</b></i> (if available) are <b>printed out (below 👇🏼) in 'read data to df' section.</b>

In [ ]:
# %env
# %who_ls
# %who
# %who int
# %pinfo <var name>

# Imports

In [ ]:
%config IPCompleter.use_jedi = False # disable jedi autocompleter (https://stackoverflow.com/a/65734178/14485040)

import project_path  # makes possible the access to `src` directory using relative path
from src.data import (
    calculate_transgression_pb,
    internal_funcs,
    transform_from_eur_to_usd,
)
from src.utils import explore_dir, make_readme_info
from src.utils import read_excel_to_pandas as r_excel
from src.utils import set_outputs_dir
from src.utils import write_pandas_to_excel as w_excel

%run init_nb.ipynb

In [ ]:
pd.options.display.max_columns = None

# INPUTS: Identify file(s) and read data to df

In [ ]:
# Explore the directory to find the file(s)
inputs_dir, files_list = explore_dir(
    path_to_dir=r"..\data\interim", file_extension="xlsx", print_files_list=True
)

In [ ]:
# Read data
df_base = r_excel(inputs_dir, "data-extended-glo-markets-and-fu-kg.xlsx")
print(
    "df_base".ljust(40, "."), f"{df_base.shape}\n".rjust(13, "."),
)

# Get list of LCIA methods and list of metadata
METHODS = r_excel(
    inputs_dir,
    "data-extended-glo-markets-and-fu-kg.xlsx",
    sheets="METHODS",
    show_readme=False,
)["METHODS"].to_list()

METADATA = r_excel(
    inputs_dir,
    "data-extended-glo-markets-and-fu-kg.xlsx",
    sheets="METADATA",
    show_readme=False,
)["METADATA"].to_list()

# Operations

## Convert `df_base.referenceProduct_price` from EUR2005 to USD2018
<div class="alert alert-block alert-info">
updated: <code>df_base</code>
</div>

In [ ]:
print(
    "There are {} unique price units in the DataFrame. And they are:".format(
        df_base.referenceProduct_priceUnit.nunique()
    )
)
pprint.pprint(list(df_base.referenceProduct_priceUnit.unique()))

In [ ]:
# change from EUR2005 to USD2018, update df_base
new_prices, new_priceUnit = transform_from_eur_to_usd(
    prices_in=df_base.referenceProduct_price,
    currency_in=df_base.referenceProduct_priceUnit[0],
    year_out=2018,
    usd_per_eur=1.1811, # average exchange rate for 2018
    adjust_inflation=True,
)

df_base.referenceProduct_price = new_prices
df_base.referenceProduct_priceUnit = new_priceUnit

## Add transgression levels (TLs) to `df_base`
<div class="alert alert-block alert-info">
    created: <code>df_base_full</code>  
</div> 

In [ ]:
df_base_full, TL_METHODS = calculate_transgression_pb(
    df_in=df_base, sharing_principle="epc",  # [METADATA + METHODS],  # equal-per-capita
)

print(
    "Created **df_base_full** dataframe is of {} shape."
    "\nAnd contains these {} new methods:".format(df_base_full.shape, len(TL_METHODS))
)
pprint.pprint(TL_METHODS)
print("\n(make sure to store these methods for later use)\n")
df_base_full.tail(5)


# NOW THIS FUNCTION RETURNS THE PASSED DF + NEW TL COLUMNS, AND LIST OF TL COLUMNS
# MAKE SURE TO CHECK THIS FURTHER

# ALSO ARE AVIALABLE DICTS WITH SOS AND CURRENT PB VALUES... USE THEM

In [ ]:
# Grouping by 'category_regrouped' and ploting the size of each group on a barh plot (in one line)
internal_funcs.plot_categories(
    df_in=df_base_full,
    groupby="category_regrouped",
    color="green",
    fontsize=12,
)

In [ ]:
# Make df of METADATA, METHODS, and TL_METHODS for later export
df_metadata = pd.DataFrame(METADATA, columns=["METADATA"])
df_methods = pd.DataFrame(METHODS, columns=["METHODS"])
df_tl_methods = pd.DataFrame(TL_METHODS, columns=["TL_METHODS"])

# OUTPUTS: Export data to excel

In [ ]:
%%time

# Set output directory
outputs_dir = set_outputs_dir(use_default=True)  # default `..\data\interim`

## Export dataframe to excel
excelName = "data-extended-added-transgression-pbs.xlsx"

df_readme = make_readme_info(
    excelName,
    "Sheet1: Extended dataframe, which also includes 9 transgression level (TL) PBs."
    "\nMETADATA: list of relevant metadata used in Sheet1."
    "\nMETHODS: list of LCIA methods used in Sheet1."
    "\nTL_METHODS: list of TL in PBs used in Sheet1."
    "\n[METHODS + METADATA + TL_METHODS have to be the only column labels in Sheet1]",
)

w_excel(
    path_to_file=outputs_dir,
    filename=excelName,
    dict_data_to_write={
        "Sheet1": df_base_full,
        "METADATA": df_metadata,
        "METHODS": df_methods,
        "TL_METHODS": df_tl_methods,
    },
    readme_info=("readme", df_readme),
    #     ExcelWriter_kwargs={"engine": "openpyxl", "encoding": "UTF-8"}
    #     startrow=0
)